In [1]:
import json
import re
from tqdm import tqdm
from evaluate import load

result_path = "/data/bhpeng/SLM-llama/tmp/results/medical/result.json"
task = 'medical'


/data/miniconda3/envs/alpaca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-17 21:19:03,156] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


{'precision': [0.9999999403953552, 1.0000001192092896], 'recall': [0.9999999403953552, 1.0000001192092896], 'f1': [0.9999999403953552, 1.0000001192092896], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}


In [2]:
def accuracy_score(label, predict):
    # Calculate accuracy for multiple-choices questions and classification
    # For generation model, we only select the first word to compare
    label = re.sub(r'[^\w\s]', ' ', label)
    predict = re.sub(r'[^\w\s]', ' ', predict)
    assert len(label.split()) == 1, "label can only contain one word"
    if len(predict.split()) == 0:
        return 0
    label = label.split()[0]
    predict = predict.split()[0]
    return int(label == predict)


In [3]:
with open(result_path) as f:
    results = json.load(f)
if task in ['history', 'finance']:
    score = 0
    for result in tqdm(results):
        label = result['label']
        output = result['output']
        score += accuracy_score(label, output)
    score = score / len(results)
    print(score)
elif task in ['medical']:
    bertscore = load("bertscore")
    outputs = []
    labels = []
    for result in results:
        labels.append(result['label'])
        outputs.append(result['output'])
    score = bertscore.compute(predictions=outputs, references=labels, lang='en')
    for k in ['precision', 'recall', 'f1']:
        score["avg_{}".format(k)] = sum(score[k]) / len(results)
    print(score['avg_precision'], score['avg_recall'], score['avg_f1'])
else:
    raise ValueError("No matched task")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8538064939690427 0.8456369483257489 0.8494767291804664


In [46]:
with open(result_path) as f:
    results = json.load(f)
if task in ['history', 'finance']:
    score = 0
    for result in tqdm(results):
        label = result['label']
        output = result['output']
        score += accuracy_score(label, output)
    score = score / len(results)
    print(score)
elif task in ['medical']:
    scores = dict(precision=[], recall=[], f1=[], sum_precision=0., sum_recall=0., sum_f1=0.)
    bertscore = load("bertscore")
    for result in tqdm(results):
        label = result['label']
        output = result['output']
        score = bertscore.compute(predictions=[output], references=[label], lang='en')
        for k in ['precision', 'recall', 'f1']:
            scores[k] += score[k]
            scores["sum_{}".format(k)] += score[k][0]
    for k in ['precision', 'recall', 'f1']:
        scores['avg_{}'.format(k)] = scores['sum_{}'.format(k)] / len(results)    
    print(result['avg_precision'], result['avg_recall'], result['avg_f1'])
else:
    raise ValueError("No matched task")

  0%|          | 0/31111 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 37/31111 [00:41<9:43:36,  1.13s/it]


KeyboardInterrupt: 